<a href="https://colab.research.google.com/github/pgallino/Organizacion-de-datos-TP1-1C2023/blob/main/spark28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=184798dd00d7190c10fc4138cf04c99f747074979131b30f91c0cd392f7809fa
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Wikipedia dataset 2c2021/contents.csv', header=True, inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


28 El porcentaje de contenidos que están publicados cuya última edición no tiene comentario para los usuarios que realizaron 1, >10 y >100 de las últimas ediciones (⭐⭐)

Filtro los que tienen datos None en los campos que me interesan.

In [ ]:
rdd = rdd.filter(lambda x: x.revisor_username != None and x.revisor_id != None)

Para poder sacar porcentajes facilmente. Serializo el estar comentado o no, con 0 si no lo está, y 1 si lo está.

In [ ]:
def serializar_comentario(x):
  if x:
    return 1
  return 0

armo registros de la forma (username, (comentado o no, contador))

In [ ]:
rdd = rdd.map(lambda x: (x.revisor_username, (serializar_comentario(x.revisor_comment) , 1)))

In [ ]:
rdd.take(5)

[('Metrónomo', (1, 1)),
 ('Traitrich', (1, 1)),
 ('Lojwe', (1, 1)),
 ('Jialxv', (1, 1)),
 ('PatriHorrillo', (1, 1))]

Sumo todos por username, quedandome (username, (cant_contenidos_sin_comentar, cant_contenidos))

In [ ]:
rdd = rdd.reduceByKey(lambda x,y : (x[0]+y[0] , x[1]+y[1])).cache()

In [ ]:
rdd.take(10)

[('Traitrich', (3696, 3868)),
 ('ClauBr', (47, 69)),
 ('TueorUe', (4, 39)),
 ('SrCerberus', (2816, 2848)),
 ('Polinizador', (6511, 18924)),
 ('KLBot2', (76523, 76523)),
 ('BenjaBot', (13291, 13292)),
 ('148LENIN', (561, 1555)),
 ('Darubrub', (29, 43)),
 ('Luisgonzalez74', (105, 148))]

Ahora, separo los usuarios por cantidad de contribuciones

In [ ]:
rdd_1 = rdd.filter(lambda x: x[1][1] == 1)

In [ ]:
rdd_10 = rdd.filter(lambda x: x[1][1] > 10)

In [ ]:
rdd_100 = rdd.filter(lambda x: x[1][1] > 100)

In [ ]:
rdd_1.take(5)

[('CarrotsMitHummus', (1, 1)),
 ('Jorgenev', (1, 1)),
 ('Fontanarr', (0, 1)),
 ('Miqalf', (1, 1)),
 ('A.Casusol', (1, 1))]

El porcentaje de contenidos sin comentar de usuarios que realizaron una contribución es del 25,7% aproximadamente.

In [ ]:
resultado_1 = (rdd_1.reduce(lambda x,y: ('resultado_1', (x[1][0] + y[1][0], x[1][1] + y[1][1]))))
100 - ((resultado_1[1][0])/(resultado_1[1][1]))*100

25.734553222361043

El porcentaje de contenidos sin comentar de usuarios que realizaron más de diez contribuciones es del 9,7% aproximadamente.

In [ ]:
resultado_10 = (rdd_10.reduce(lambda x,y: ('resultado_10', (x[1][0] + y[1][0], x[1][1] + y[1][1]))))
100 - ((resultado_10[1][0])/(resultado_10[1][1]))*100

9.675239504198728

El porcentaje de contenidos sin comentar de usuarios que realizaron más de cien contribuciones es del 9,2% aproximadamente.

In [ ]:
resultado_100 = (rdd_100.reduce(lambda x,y: ('resultado_100', (x[1][0] + y[1][0], x[1][1] + y[1][1]))))
100 - ((resultado_100[1][0])/(resultado_100[1][1]))*100

9.233210456511898